# 1. Multivariate Linear Regression 복습
 - 복수의 정보를 통해 하나의 추측값을 계산한다.
 - 세 개의 쪽지시험 성적을 통해 기말 성적을 예측한다.
 - 학습 시킬 수록 점점 Cost(Loss)는 줄어들었다.

<br>
<hr>
<br>

# 2. Data in the Real World
### (1) Data in the Real World
 - 복잡한 머신러닝 모델을 학습하려면 엄청난 양의 데이터가 필요하다.
 - 대부분의 데이터셋은, 적어도 수십만 개의 데이터를 제공한다.

<br>

### (2) Problem
 - 엄청난 양의 데이터를 한 번에 학습시킬 수 없다. 느리고, 하드웨어적으로 불가능
 - 일부분의 데이터로만 학습시키면 어떨까?

<br>
<hr>
<br>


# 3. "Minibatch" Gradient Descent 이론
### (1) Minibatch란?
 - 데이터의 양이 엄청나게 많을 때, 전체 데이터를 균일하게 나눠서 미니배치 하나하나씩 학습하자!
 - 전체 데이터 => Minibatch1 + Minibatch2 + Minibatch3 + Minibatch4 + Minibatch5

<br>

### (2) Minibatch 장점
 - 한꺼번에 Gradient Descent 하는 것이 아닌, 각 Minibatch마다 cost를 계산해서 Gradient Descent를 할 수 있기 때문에 컴퓨터에 무리가 덜 간다.
 - 한 번의 업데이트마다 계산할 cost 양은 줄어들고, 업데이트의 주기가 빨라진다.
 
<br>

### (3) Minibatch 단점
 - 모델의 cost를 계산할 때 '전체 데이터'를 쓴 것이 아니므로, 가끔 잘못된 방향으로 업데이트를 할 수도 있다.
 - 기존 Gradient Descent보다 cost가 거칠게 줄어든다.


<br>
<hr>
<br>


# 4. PyTorch Dataset and DataLoader 사용법
### (1) PyTorch Dataset
  - PyTorch Dataset : PyTorch에서 제공하는 모듈로, 이 모듈을 상속해 새로운 클래스를 만들어서 우리가 원하는 데이터를 만들 수 있다.
  - __len__() : 데이터 셋의 총 데이터 수 반환
  - __getitem__() : 인덱스 idx에 상응하는 입출력 데이터 반환
  
  <br>
  
### (2) DataLoader
 - PyTorch Dataset으로 데이터 셋을 만들면 DataLoader를 이용할 수 있다.
 - 지정값 1 : 데이터 셋
 - 지정값 2 : Minibatch 크기 (통상적으로 2의 제곱수로 설정한다. 16, 32, 64, 128, 256, 512 ...)
 - 지정값 3 : shuffle = True : Epoch 마다 데이터 셋을 섞어서, 데이터가 학습되는 순서를 바꾼다. (모델이 데이터셋의 '순서'를 외우는 치팅을 안하게 방지!)
 - enumerate(dataloader) : minibatch의 인덱스와 데이터를 받아온다.
 - len(dataloader) : 한 epoch 당 minibatch 개수

In [1]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                         [93, 88, 93],
                         [89, 91, 90],
                         [96, 98, 100],
                         [79, 66, 70]]
        self.y_data = [[152], [185], [150], [180], [196], [142]]
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        t = torch.FloatTensor(self.y_data[idx])
    
        return x, t

In [3]:
dataset = CustomDataset()

In [4]:
from torch.utils.data import DataLoader
 
dataloader = DataLoader(
  dataset,
  batch_size = 2,
  shuffle = True,
 )

In [5]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # nn.Linear에 입력 차원과 출력 차원을 설정한다.

    def forward(self, x):
        return self.linear(x) # forward 함수에 hypothesis 계산을 어떻게 할지 설정한다.

In [6]:
model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [7]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        prediction = model(x_train)
    
        cost = F.mse_loss(prediction, y_train)
    
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        print('Epoch {:4d}/{} Batch {}/{} Cost : {:.6f}'.format(epoch, nb_epochs, batch_idx, len(dataloader), cost.item()))

Epoch    0/20 Batch 0/3 Cost : 18785.634766
Epoch    0/20 Batch 1/3 Cost : 7071.236816
Epoch    0/20 Batch 2/3 Cost : 19.404148
Epoch    1/20 Batch 0/3 Cost : 3417.357666
Epoch    1/20 Batch 1/3 Cost : 113.411949
Epoch    1/20 Batch 2/3 Cost : 453.736298
Epoch    2/20 Batch 0/3 Cost : 182.580704
Epoch    2/20 Batch 1/3 Cost : 248.469604
Epoch    2/20 Batch 2/3 Cost : 3830.438232
Epoch    3/20 Batch 0/3 Cost : 1291.367676
Epoch    3/20 Batch 1/3 Cost : 47.875641
Epoch    3/20 Batch 2/3 Cost : 2960.111328
Epoch    4/20 Batch 0/3 Cost : 752.850586
Epoch    4/20 Batch 1/3 Cost : 1772.120728
Epoch    4/20 Batch 2/3 Cost : 12.287948
Epoch    5/20 Batch 0/3 Cost : 803.343933
Epoch    5/20 Batch 1/3 Cost : 1795.969482
Epoch    5/20 Batch 2/3 Cost : 1.043649
Epoch    6/20 Batch 0/3 Cost : 1776.689697
Epoch    6/20 Batch 1/3 Cost : 11.376652
Epoch    6/20 Batch 2/3 Cost : 362.503906
Epoch    7/20 Batch 0/3 Cost : 1657.966064
Epoch    7/20 Batch 1/3 Cost : 857.620972
Epoch    7/20 Batch 2/3 Cost 